https://www.youtube.com/watch?v=OSqIP-mOWOI&list=PL_lsbAsL_o2CTlGHgMxNrKhzP97BaG9ZN&index=4&pp=iAQB

In [3]:
import torch

class TinyModel(torch.nn.Module):
    def __init__(self):
        super(TinyModel, self).__init__()
        
        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()
print(tinymodel)

print('\n\nLayer2:')
print(tinymodel.linear2)

print('\n\nModel params:')
for param in tinymodel.parameters():
    print(param.shape)
    print(param)

print('\n\nLayer2 params:')
for param in tinymodel.linear2.parameters():
    print(param.shape)
    print(param)

TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Layer2:
Linear(in_features=200, out_features=10, bias=True)


Model params:
torch.Size([200, 100])
Parameter containing:
tensor([[-0.0858, -0.0893, -0.0389,  ...,  0.0647, -0.0460,  0.0464],
        [ 0.0979, -0.0645,  0.0183,  ..., -0.0590, -0.0326, -0.0422],
        [-0.0435,  0.0732,  0.0272,  ..., -0.0437,  0.0120,  0.0695],
        ...,
        [-0.0101,  0.0499, -0.0034,  ..., -0.0111, -0.0587, -0.0748],
        [-0.0477,  0.0664,  0.0539,  ...,  0.0105, -0.0693, -0.0506],
        [ 0.0350,  0.0160,  0.0005,  ...,  0.0224, -0.0477,  0.0353]],
       requires_grad=True)
torch.Size([200])
Parameter containing:
tensor([ 0.0182, -0.0069,  0.0473,  0.0671,  0.0877,  0.0522, -0.0824,  0.0560,
         0.0477, -0.0119, -0.0361,  0.0881, -0.0270, -0.0897, -0.0238,  0.0843,
        -0.0383, -

## Common Layer Types

In [6]:
lin = torch.nn.Linear(3,2)
x = torch.rand(1,3)
print('Input:')
print(x)

print('\n\nWeight and Bias Parameters:')
for param in lin.parameters():
    print(param.shape)
    print(param)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.1954, 0.1121, 0.4606]])


Weight and Bias Parameters:
torch.Size([2, 3])
Parameter containing:
tensor([[-0.5050, -0.5167,  0.4285],
        [ 0.0127,  0.3596, -0.2269]], requires_grad=True)
torch.Size([2])
Parameter containing:
tensor([-0.5326, -0.4200], requires_grad=True)


Output:
tensor([[-0.4918, -0.4817]], grad_fn=<AddmmBackward0>)


### Convolutional Layers

In [8]:
import torch.functional as F

class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        # 1 input image channel (black & white), 6 output channels, 3x3 square convolution kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = torch.nn.Linear(16*6*6, 120) # 6*6 from image dimension
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # If the size is a square, you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

### Recurrent Layers

In [9]:
class LSTMTagger(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

## Other Layers and Functions

In [10]:
my_tensor = torch.rand(1,6,6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

tensor([[[0.9057, 0.3819, 0.6223, 0.1159, 0.6162, 0.1379],
         [0.7229, 0.8933, 0.8230, 0.1987, 0.7447, 0.7009],
         [0.5246, 0.8361, 0.8915, 0.9797, 0.0393, 0.3456],
         [0.4692, 0.7562, 0.3610, 0.6188, 0.5126, 0.4137],
         [0.5575, 0.3412, 0.1996, 0.7662, 0.4150, 0.7437],
         [0.8331, 0.1640, 0.2648, 0.3901, 0.9983, 0.8148]]])
tensor([[[0.9057, 0.9797],
         [0.8331, 0.9983]]])


In [11]:
my_tensor = torch.rand(1,4,4) * 20 + 5
print(my_tensor)

print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)
print(normed_tensor.mean())

tensor([[[14.9145,  8.4515, 20.2624, 18.6193],
         [14.2996, 11.0874, 23.5475, 10.1099],
         [10.3311, 22.7006,  5.0069, 15.2135],
         [14.8861,  5.8449, 11.4248,  6.2728]]])
tensor(13.3108)
tensor([[[-0.1426, -1.5664,  1.0355,  0.6735],
         [-0.0870, -0.6926,  1.6565, -0.8769],
         [-0.4579,  1.4416, -1.2755,  0.2918],
         [ 1.4053, -1.0015,  0.4839, -0.8876]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(-2.2352e-08, grad_fn=<MeanBackward0>)


In [12]:
my_tensor = torch.rand(1,4,4)
print(my_tensor)

dropout = torch.nn.Dropout(p=0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.4462, 0.5660, 0.3722, 0.7879],
         [0.5048, 0.0812, 0.9505, 0.5933],
         [0.0241, 0.7332, 0.6322, 0.7899],
         [0.8347, 0.0732, 0.8985, 0.0033]]])
tensor([[[0.0000, 0.9434, 0.6203, 1.3131],
         [0.8414, 0.1353, 0.0000, 0.9888],
         [0.0402, 1.2219, 1.0536, 0.0000],
         [1.3911, 0.1220, 0.0000, 0.0000]]])
tensor([[[0.7437, 0.0000, 0.6203, 0.0000],
         [0.8414, 0.0000, 1.5842, 0.9888],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.1220, 0.0000, 0.0055]]])
